# Algoritmos de búsqueda
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

Un algoritmo de búsqueda toma un problema de búsqueda como entrada y retorna una solución, o una indicación de falla.

La idea es buscar un camino que llegue al estado objetivo. Para ello, vamos a construir un árbol que irá avanzando por los estados del grafo hasta llegar al estado objetivo.

![arbol_de_hanoi](./img/tree_hanoi.png)

Cada nodo del árbol corresponde a un **estado** y las aristas corresponden a una **acción**. Es importante destacar que el árbol **NO** es el grafo de estados. El grafo describe todo el conjunto de estados y las acciones que permiten pasar de un estado a otro. El árbol describe el camino entre estos estados para alcanzar el objetivo.

Para poder aplicar los algoritmos, debemos definir las estructuras de datos necesarias para hacer seguimiento del árbol.

## Nodos del árbol

Los nodos del árbol están representados por los siguientes componentes:

- **State**: El estado del espacio de estados que corresponde al nodo.
- **Node Parent**: El nodo en el árbol de búsqueda que generó este nodo.
- **Action**: La acción que se aplicó al nodo padre para generar este nodo.
- **Path-Cost**: El costo de un camino desde el nodo inicial hasta este nodo.

Definamos el **Problema** de la Torre de Hanoi:

In [1]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi

initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)

problem = ProblemHanoi(initial=initial_state, goal=goal_state)

Empezamos con la primera estructura, que implementamos con la clase `NodeHanoi`. Esta clase tiene implementados los siguientes:

**Atributos:**
* `state`: El estado que el nodo contiene. Representa un estado particular de ubicación de los discos.
* `parent`: Es el nodo padre de este nodo. Si el nodo es la raíz, es `None`.
* `action`: La acción que se aplicó al nodo padre para llegar a este nodo. Si es la raíz, es `None`.
* `path_cost`: El costo del camino desde la raíz del árbol hasta este nodo.
* `depth`: La profundidad del árbol en la que se encuentra el nodo. Si es la raíz, es cero; si es un hijo de la raíz, es igual a 1, y así sucesivamente.

**Métodos:**
* `child_node()`: Genera un nodo hijo a partir de una acción.
* `expand()`: Expande la frontera de este nodo, devolviendo los nodos hijos al aplicar `expand`.
* `solution()`: Retorna en una lista la secuencia de acciones que van desde la raíz hasta este nodo.
* `path()`: Retorna una lista de nodos que van desde la raíz hasta este nodo.
* `generate_solution_for_simulator()`: Este método permite obtener una salida para simular con PyGame. En otro notebook profundizaremos más en este aspecto.

Además, tiene implementados métodos que nos permiten hacer diferentes operaciones en Python:

* Podemos comparar dos nodos para ver si son iguales (haciendo `node1 == node2`)
* Podemos preguntar si un nodo es mayor que otro (haciendo `node1 > node2`), lo que significa si el costo acumulado de un nodo es mayor que otro.
* Tenemos una representación en cadena del estado, por lo que cuando hacemos `print()` se observa el estado dentro del nodo con el texto `<Node >`.
* También podemos obtener un hash del estado con `hash(estado)`


In [2]:
from aima_libs.tree_hanoi import NodeHanoi

In [ ]:
# Definimos la raíz del árbol
root = NodeHanoi(state=initial_state)

In [4]:
print("El arbol tiene como raíz a:")
print(root)

El arbol tiene como raíz a:
<Node HanoiState: 5 4 3 2 1 |  | >


Desde un nodo y con el problema definido, podemos encontrar la frontera, que es la separación entre el grafo que ya ha sido explorado por el algoritmo de búsqueda y aquel que aún no ha sido explorado.

![frontera_en_arbol_de_hanoi](./img/tree_hanoi_frontier.png)

### Expandir la frontera del nodo raíz

Expandimos la frontera del nodo raíz:


In [5]:
lista_nodos_fronteras = root.expand(problem=problem)

In [6]:
for nodos in lista_nodos_fronteras:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 2 |  | 1>


Los estados que corresponden a estos nuevos nodos son:

![state](./img/state_hanoi5.png)

Por lo tanto, el árbol para este problema se verá así:

![tree](./img/tree_hanoi2.png)

Es decir, se generan dos nodos nuevos con los siguientes estados:

- El disco verde (el más pequeño) se mueve a la varilla del medio.
- El disco verde (el más pequeño) se mueve a la varilla derecha.

Quedémonos con el nodo cuyo estado tiene el disco verde en la varilla del medio:

In [7]:
next_node = lista_nodos_fronteras[0]

Ahora, veamos su estado:

In [8]:
next_node.state

HanoiState: 5 4 3 2 | 1 | 

Vemos quién es el padre, que debería ser la raíz:

In [9]:
next_node.parent

<Node HanoiState: 5 4 3 2 1 |  | >

El costo acumulado desde la raíz hasta este nodo es:

In [10]:
next_node.path_cost

1.0

Y la profundidad en la que nos encontramos en el árbol:

In [11]:
next_node.depth

1

### Expandir la frontera del nodo siguiente


Expandimos ahora la frontera de este nodo:

In [12]:
lista_nodos_fronteras2 = next_node.expand(problem=problem)

In [13]:
for nodos in lista_nodos_fronteras2:
    print(nodos)

<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>


Veamos cómo quedó ahora el árbol:

![tree](./img/tree_hanoi3.png)

Observamos lo siguiente: se generaron tres nuevos nodos en la frontera. De estos, hay dos que llaman la atención.

- Hay un nodo cuyo estado es igual al del nodo padre. Esto ocurre porque, si movemos el disco verde de vuelta a la varilla de la izquierda, *"retornamos"* al estado inicial.
- Hay un nodo que tiene el mismo estado que el segundo nodo obtenido del padre.

Esto es importante de destacar: **múltiples nodos pueden tener el mismo estado**, pero el costo para llegar a ese nodo y la secuencia de acciones desde la raíz hasta ese nodo serán diferentes.

Veamos ahora el nodo cuyo estado no está repetido:

In [14]:
next_node2 = lista_nodos_fronteras2[0]

Vemos su estado:

In [15]:
next_node2.state

HanoiState: 5 4 3 | 1 | 2

Vemos quién es el padre:

In [16]:
next_node2.parent

<Node HanoiState: 5 4 3 2 | 1 | >

El costo acumulado desde la raíz hasta este nodo es:

In [17]:
next_node2.path_cost

2.0

Y la profundidad en la que estamos en el árbol:

In [18]:
next_node2.depth

2

Veamos el camino desde la raíz hasta este nodo:

In [19]:
for nodos in next_node2.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 1 | 2>


Y las acciones que se aplicaron desde el inicio hasta este nodo:

In [20]:
for nodos in next_node2.solution():
    print(nodos)

Move disk 1 from 1 to 2
Move disk 2 from 1 to 3


## Colas

La pregunta es: ¿cómo hacemos para elegir cuál nodo seleccionar para explorar la frontera? Para ello necesitamos una estructura de datos que nos permita explorar la frontera. Esta estructura es vital para el algoritmo de búsqueda, ya que nos permite seleccionar qué nodo vamos a expandir primero. Como vimos en los videos, elegir el tipo de estructura para expandir es lo que define el tipo de algoritmo.

La frontera se expande usando **colas**, y tenemos tres tipos:

- Una cola **FIFO** (Primero entra, primero sale) que toma los nodos en el mismo orden en que se agregan.
- Una cola **LIFO** (Último en entrar, primero en salir, o "stack") que quita el nodo más reciente.
- Una **cola prioritaria** que quita primero los nodos con el mínimo costo, de acuerdo con una función de evaluación `f()`.

### FIFO

Veamos una implementación de cola FIFO usando una lista:

In [21]:
fifo = []

La implementación debe incorporar las siguientes funciones:

- **Add(Frontier)**: Inserta el nodo en su correspondiente lugar de la cola. En el caso de la FIFO, inserta los nodos en el orden en que van llegando, utilizando el método `insert()`.

In [22]:
fifo.insert(0, lista_nodos_fronteras[0])
fifo.insert(0, lista_nodos_fronteras[1])

for nodos in fifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 | 1 | >


- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En el caso de la implementación con una lista, podemos preguntar si la cantidad de elementos es cero.

In [23]:
print("La cola está vacia?")
if len(fifo) == 0:
    print("La cola esta vacía")
else:
    print("La cola tiene elementos")

La cola está vacia?
La cola tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con las listas, podemos usar el método `pop()`.

In [24]:
new_node = fifo.pop()
print(f"El nodo que sacamos es {new_node}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 | 1 | >


Podemos ver que, una vez sacado el nodo, el mismo ya no está en la fila, y que particularmente sacamos el primer nodo que entró:

In [ ]:
print("Los nodos que quedan en la fila son:")
print(fifo)

Los nodos que quedan en la fila son:
[<Node HanoiState: 5 4 3 2 |  | 1>]


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [ ]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

# Insertamos los nodos de frontera en el orden que nos fue presentado:
fifo.insert(0, lista_nodos_fronteras2[0])
fifo.insert(0, lista_nodos_fronteras2[1])
fifo.insert(0, lista_nodos_fronteras2[2])

In [27]:
for nodos in fifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 2 |  | 1>


- **Top(frontier)**: Podemos ver cuál es nuestro siguiente nodo sin sacarlo:

In [28]:
print(f"El siguiente nodo que podemos sacar es {fifo[-1]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 |  | 1>


Este es el segundo nodo que introdujimos cuando agregamos la frontera de la raíz. Como vemos, se respeta el principio *"Primero entra, primero sale"*.

### LIFO

Ahora veamos cómo podemos implementar un **stack**. Esto también lo podemos hacer con una lista.

In [29]:
lifo = []

- **Add(Frontier)**: Inserta el nodo en su correspondiente lugar de la cola. En el caso de la LIFO, se inserta de forma apilada para que el último nodo que se inserte, esté listo para salir. Esto lo hacemos usando `append()`:

In [30]:
lifo.append(lista_nodos_fronteras[0])
lifo.append(lista_nodos_fronteras[1])

for nodos in lifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 2 |  | 1>


> ⚠️ **Nota**: Los nodos están ordenados de forma inversa al caso de la fila FIFO.

- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En el caso de la implementación con una lista, podemos preguntar si la cantidad de elementos es cero.

In [31]:
print("El stack está vacio?")
if len(lifo) == 0:
    print("El stack esta vacio")
else:
    print("El stack tiene elementos")

El stack está vacio?
El stack tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con las listas, usamos el método `pop()`.

In [32]:
new_node = lifo.pop()
print(f"El nodo que sacamos es {new_node}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 |  | 1>


Podemos ver que, una vez sacado el nodo, el mismo ya no está en la fila, y que particularmente sacamos el último nodo que entró:

In [33]:
print("Los nodos que quedan en la fila son:")
print(lifo)

Los nodos que quedan en la fila son:
[<Node HanoiState: 5 4 3 2 | 1 | >]


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [34]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

lifo.append(lista_nodos_fronteras2[0])
lifo.append(lista_nodos_fronteras2[1])
lifo.append(lista_nodos_fronteras2[2])

In [35]:
for nodos in lifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 | 1 | >


- **Top(frontier)**: Podemos ver cuál es nuestro siguiente nodo sin sacarlo:

In [36]:
print(f"El siguiente nodo que podemos sacar es {lifo[-1]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 | 1 | >


Este es el último nodo que introdujimos cuando agregamos la frontera del nodo que sacamos de la cola. Como vemos, se respeta el principio *"Último entra, primero sale"*.

### Cola prioritaria

Por último, nos queda ver la cola prioritaria. Para ello, usaremos la librería `queue` que viene por defecto en Python. [Esta librería](https://docs.python.org/3/library/queue.html) implementa varias colas (incluidas las colas FIFO y LIFO). Particularmente, vamos a usar la clase `PriorityQueue`.

In [37]:
from queue import PriorityQueue

priority_queue = PriorityQueue()

- **Add(Frontier)**: Para esta cola, se debe pasar el nodo que queremos encolar, pero además debemos pasar un número que defina la prioridad en forma de tupla. Esta cola siempre extrae el nodo con el valor de prioridad más bajo.

In [38]:
priority_queue.put((4, lista_nodos_fronteras[0]))
priority_queue.put((2, lista_nodos_fronteras[1]))

Como vimos en clase, el valor que podemos guardar puede ser:

- El costo hasta ese nodo (`g(n)`),
- Una heurística (`h(n)`),
- O una combinación de ambas (`f(n) = g(n) + h(n)`).

En este caso usamos valores arbitrarios solo para ver el funcionamiento.

- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. Podemos usar el método `empty()`:

In [39]:
print("La cola prioritaria está vacia?")
if priority_queue.empty():
    print("La cola prioritaria esta vacia")
else:
    print("La cola prioritaria tiene elementos")

La cola prioritaria está vacia?
La cola prioritaria tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola usando el método `get()`. El nodo extraído será el de menor prioridad.

In [40]:
priority_value, new_node = priority_queue.get()
print(f"El nodo que sacamos es {new_node}, cuyo valor de prioridad es {priority_value}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 |  | 1>, cuyo valor de prioridad es 2


Si expandimos la frontera del nuevo nodo, podemos agregar los nuevos nodos generados a la cola:

In [41]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

priority_queue.put((1, lista_nodos_fronteras2[0]))
priority_queue.put((24, lista_nodos_fronteras2[1]))
priority_queue.put((54, lista_nodos_fronteras2[2]))

- **Top(frontier)**: Para ver el siguiente nodo sin sacarlo, podemos acceder directamente a la lista interna queue (aunque esto no es lo más recomendable en producción):

In [42]:
print(f"El siguiente nodo que podemos sacar es {priority_queue.queue[0][-1]}, el cual tiene prioridad {priority_queue.queue[0][0]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 | 2 | 1>, el cual tiene prioridad 1


#### Cola prioritaria de AIMA

También tenemos implementada una cola prioritaria que permite usar una función de evaluación directamente, lo que simplifica bastante la gestión de prioridades. Esta implementación forma parte de la biblioteca `aima`.

In [43]:
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

priority_queue2 = AimaPriorityQueue()

Con esta cola, podemos:

- Definir si la prioridad es por el menor o mayor valor (`order='min'` o `order='max'`).
- Pasar una función `f()` que, dado un nodo, devuelva su prioridad.

Por ejemplo, para probar su funcionamiento, podemos definir una función que devuelva una prioridad aleatoria:

In [44]:
import random

def priority_func(x):
    return random.randint(1, 1000)

In [45]:
priority_queue2 = AimaPriorityQueue(order='min', f=priority_func)

- **Add(Frontier)**: Se puede usar el método `append()` para agregar un nodo individual:

In [46]:
priority_queue2.append(lista_nodos_fronteras[0])
priority_queue2.append(lista_nodos_fronteras[1])

También se puede usar `extend()` para agregar varios nodos de una sola vez:

In [47]:
priority_queue2.extend(lista_nodos_fronteras2)

Para ver los elementos dentro de la cola:

In [48]:
priority_queue2.heap.queue

[(31, <Node HanoiState: 5 4 3 | 2 | 1>),
 (226, <Node HanoiState: 5 4 3 2 1 |  | >),
 (119, <Node HanoiState: 5 4 3 2 |  | 1>),
 (582, <Node HanoiState: 5 4 3 2 | 1 | >),
 (889, <Node HanoiState: 5 4 3 2 | 1 | >)]

- **Is-empty(frontier)**: Verifica si la cola está vacía:

In [49]:
print("La cola prioritaria está vacia?")
if len(priority_queue2) == 0:
    print("La cola prioritaria esta vacia")
else:
    print("La cola prioritaria tiene elementos")

La cola prioritaria está vacia?
La cola prioritaria tiene elementos


- **Pop(frontier)**: Quita el nodo con menor prioridad:

In [50]:
priority_value, new_node = priority_queue2.pop()
print(f"El nodo que sacamos es {new_node}, cuyo valor de prioridad es {priority_value}")

El nodo que sacamos es <Node HanoiState: 5 4 3 | 2 | 1>, cuyo valor de prioridad es 31


- **Top(frontier)**: Podemos ver el siguiente nodo que será extraído con el método `peek()`:

In [51]:
print(f"El siguiente nodo que podemos sacar es {priority_queue2.peek()[-1]}, el cual tiene prioridad {priority_queue2.peek()[0]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 |  | 1>, el cual tiene prioridad 119


### Conclusión

Todo lo que hemos visto hasta ahora *—desde cómo definimos el problema hasta cómo se define la cola—* son ejemplos de implementación. Estos sirven como base, pero cada persona o proyecto es libre de usar estas soluciones o implementar las propias, de acuerdo a las necesidades del problema o del entorno en el que se esté trabajando.

----

## Búsqueda Primero en Anchura

Con todo lo que fuimos implementando, ahora podemos aplicar un algoritmo de búsqueda. Observá todo lo que tuvimos que definir e implementar previamente para poder llegar hasta este punto.

Vamos a implementar el algoritmo de **búsqueda primero en anchura (Breadth-First Search)**, tal como vimos en el video. Este algoritmo comienza desde la raíz y va expandiendo todos los nodos nivel por nivel utilizando una cola **FIFO**.

![breadth_first_search](./img/breadth_first_search.png)

### Probamos con menos discos

Empecemos con menos discos para probar cómo implementarlo y luego pasamos al caso con 5 discos:

In [ ]:
# Inicializamos el problema
initial_state = StatesHanoi([3, 2, 1], [], [], max_disks=3)
goal_state = StatesHanoi([], [], [3, 2, 1], max_disks=3)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

find_solution = False

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

Encontramos la solución


Una vez encontrada, podemos analizar el último nodo del árbol y ver cuánto costó llegar a la solución:

In [53]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')
print(f'Profundidad del arbol donde se encontró la solución: {last_node.depth}')

Longitud del camino de la solución: 7.0
Profundidad del arbol donde se encontró la solución: 7


Probemos el mismo algoritmo con 4 discos:

In [ ]:
initial_state = StatesHanoi([4, 3, 2, 1], [], [], max_disks=4)
goal_state = StatesHanoi([], [], [4, 3, 2, 1], max_disks=4)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

find_solution = False

while len(frontier) != 0:
    node = frontier.pop()
    if problem.goal_test(node.state):
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    for next_node in node.expand(problem):
        frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

KeyboardInterrupt: 

Este algoritmo puede tardar bastante tiempo (se puede interrumpir la ejecución si se desea), ya que ocurre un fenómeno que analizamos previamente al ver el árbol de búsqueda:

![tree](./img/tree_hanoi3.png)

Cuando expandimos un nodo de la frontera, los nuevos nodos pueden tener estados que ya fueron explorados en pasos anteriores, lo que hace que el algoritmo repita caminos y entre en bucles (aunque no infinitos, sí innecesariamente largos).

### Versión con control de estados explorados

Vamos a crear una variante del algoritmo que consume más memoria, pero evita explorar estados repetidos. Usamos para eso un `set`, estructura ideal para verificar rápidamente si un estado ya fue explorado:

In [ ]:
initial_state = StatesHanoi([4, 3, 2, 1], [], [], max_disks=4)
goal_state = StatesHanoi([], [], [4, 3, 2, 1], max_disks=4)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]

explored = set()  # Almacena los estados que ya fueron explorados

find_solution = False
node_explored = 0

while len(frontier) != 0:
    node = frontier.pop()
    node_explored += 1
    
    explored.add(node.state) # Agregamos el estado al set de explorados
    
    if problem.goal_test(node.state):
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si el estado del nodo no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

Encontramos la solución


Una vez que se encuentra la solución, podemos ver:

In [56]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')
print(f'Profundidad del arbol donde se encontró la solución: {last_node.depth}')

Longitud del camino de la solución: 15.0
Profundidad del arbol donde se encontró la solución: 15


Como todos los pasos tienen el mismo costo, la **búsqueda primero en anchura** garantiza que el camino encontrado es el más corto posible en términos de número de acciones. Es decir, el algoritmo encontró la solución en la mínima profundidad del árbol.

Además, podemos analizar el desempeño del algoritmo:

In [57]:
print(node_explored, "nodos se expandieron, correspondientes a", len(explored),"estados diferentes y", len(frontier), "nodos quedaron en la frontera")

216 nodos se expandieron, correspondientes a 71 estados diferentes y 52 nodos quedaron en la frontera


Podemos visualizar el camino desde el estado inicial hasta la solución:

In [58]:
for nodos in last_node.path():
    print(nodos)

<Node HanoiState: 4 3 2 1 |  | >
<Node HanoiState: 4 3 2 | 1 | >
<Node HanoiState: 4 3 | 1 | 2>
<Node HanoiState: 4 3 |  | 2 1>
<Node HanoiState: 4 | 3 | 2 1>
<Node HanoiState: 4 1 | 3 | 2>
<Node HanoiState: 4 1 | 3 2 | >
<Node HanoiState: 4 | 3 2 1 | >
<Node HanoiState:  | 3 2 1 | 4>
<Node HanoiState:  | 3 2 | 4 1>
<Node HanoiState: 2 | 3 | 4 1>
<Node HanoiState: 2 1 | 3 | 4>
<Node HanoiState: 2 1 |  | 4 3>
<Node HanoiState: 2 | 1 | 4 3>
<Node HanoiState:  | 1 | 4 3 2>
<Node HanoiState:  |  | 4 3 2 1>


Y las acciones que el agente debería aplicar para alcanzar el estado objetivo:

In [59]:
for act in last_node.solution():
    print(act)

Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 2
Move disk 4 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3


### Creando una función y resolviendo con 5 discos

Ahora que tenemos una implementación funcional, vamos a crear una función que aplique el algoritmo de **búsqueda primero en anchura (Breadth-First Search)**, y que retorne tanto la solución encontrada como algunas métricas de ejecución:

In [ ]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    frontier = [NodeHanoi(problem.initial)] # Cola FIFO con el nodo inicial
    explored = set() # Conjunto de estados ya visitados

    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        explored.add(node.state) # Verificamos si llegamos al objetivo
        
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la frontera los nodos sucesores que no hayan sido visitados
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                frontier.insert(0, next_node)

    # Si no se encuentra solución, devolvemos métricas igualmente
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

Ejecutamos con 5 discos:

In [61]:
solution, metrics = breadth_first_search(number_disks=5)

Veamos algunas métricas:

In [62]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 1351
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


Para ver el camino completo desde el estado inicial hasta la solución:

In [63]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

Y las acciones que el agente debería aplicar para llegar al objetivo:

In [64]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


## Medición de performance

Por último, si nos preguntamos cómo medir la performance del algoritmo de búsqueda, Python nos ofrece algunas herramientas de *profiling*.

### Medir el tiempo de ejecución
Podemos utilizar `%%timeit` en notebooks de Jupyter para medir el tiempo de ejecución promedio de una celda:

In [65]:
%%timeit 
solution, metrics = breadth_first_search(number_disks=5)

61.2 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


En una prueba típica, por ejemplo, podemos observar que la ejecución demora unos 60 milisegundos. Este valor dependerá de la máquina utilizada y solo será significativo si se compara con otros algoritmos ejecutados en las mismas condiciones.

### Medir el uso de memoria

Medir el uso de memoria es un poco más complejo. Una forma sencilla es monitorear el pico de consumo de memoria RAM durante la ejecución. Para esto podemos usar la biblioteca `tracemalloc`:

In [66]:
import tracemalloc

tracemalloc.start()

solution, metrics = breadth_first_search(number_disks=5)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 1.61 [MB]


Con esta estructura, tenemos un algoritmo funcional, evaluable en rendimiento y reutilizable. Desde acá podemos comparar esta implementación de búsqueda con otras variantes como:

- Búsqueda en profundidad primero
- Búsqueda con costo uniforme 
- Búsqueda Greedy
- Búsqueda A*